# Install Package, Library, and Datasets

In [1]:
!nvidia-smi

Wed Feb  4 03:15:15 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:87:00.0 Off |                    0 |
| N/A   34C    P0             65W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [60]:
!pip install gdown nltk transformers transformers[torch] g2p_en datasets soundfile evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!gdown 1Uw4lAW28xDh5tijF4on32CtvxsatzEPi

Downloading...
From (original): https://drive.google.com/uc?id=1Uw4lAW28xDh5tijF4on32CtvxsatzEPi
From (redirected): https://drive.google.com/uc?id=1Uw4lAW28xDh5tijF4on32CtvxsatzEPi&confirm=t&uuid=975e974f-8433-4637-9eb7-0c6f90ceb6e7
To: /workspace/LibriSpeech_final.zip
100%|███████████████████████████████████████| 2.45G/2.45G [00:18<00:00, 131MB/s]


In [4]:
!apt update -y && apt install unzip

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2329 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3640 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1296 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6288 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]33m
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]       
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [38.5 kB]
Get:12 http://archiv

In [76]:
!apt install zip -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 136 not upgraded.
Need to get 176 kB of archives.
After this operation, 544 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 176 kB in 1s (217 kB/s)0m33m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package zip.
(Reading database ... 24153 files and directories currently installed.)
Preparing to unpack .../zip_3.0-12build2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking zip (3.0-12build2) ...
7Progress: [ 40%] [#######################...................................] 8Setting up zip (3.0-12build2) .

In [5]:
!unzip LibriSpeech_final.zip

Archive:  LibriSpeech_final.zip
   creating: LibriSpeech_final/
   creating: LibriSpeech_final/train-clean/
   creating: LibriSpeech_final/train-clean/4441/
   creating: LibriSpeech_final/train-clean/4441/76263/
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0014.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0019.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0029.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0044.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0025.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/76263.trans.txt  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0026.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0023.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0043.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0030.wav  
  inflating: LibriSpeech_final/t

# Import Library

In [61]:
import torch
import os
import soundfile as sf
from g2p_en import G2p
from datasets import Dataset
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
    BatchEncoding
)
from dataclasses import dataclass
from typing import Dict, List, Union
from torch.nn.utils.rnn import pad_sequence

import evaluate
import numpy as np

import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

# Fine Tuning

## Change Device to Cuda

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


## Load Pretrained Model

In [8]:
MODEL_ID = "moxeeeem/wav2vec2-finetuned-pronunciation-correction"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(device)

preprocessor_config.json:   0%|          | 0.00/257 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/424 [00:00<?, ?it/s]

## Utilities Function for Preprocessing Data

In [20]:
g2p = G2p()

ARPABET_TO_IPA = {
    'AA': 'ɑ', 'AE': 'æ', 'AH': 'ʌ', 'AO': 'ɔ', 'EH': 'ɛ',
    'ER': 'ɝ', 'IH': 'ɪ', 'IY': 'i', 'UH': 'ʊ', 'UW': 'u',
    'AW': 'aʊ', 'AY': 'aɪ', 'EY': 'eɪ', 'OW': 'oʊ', 'OY': 'ɔɪ',
    'P': 'p', 'B': 'b', 'T': 't', 'D': 'd', 'K': 'k', 'G': 'g',
    'CH': 'tʃ', 'JH': 'dʒ', 'F': 'f', 'V': 'v', 'TH': 'θ', 'DH': 'ð',
    'S': 's', 'Z': 'z', 'SH': 'ʃ', 'ZH': 'ʒ', 'HH': 'h', 'M': 'm',
    'N': 'n', 'NG': 'ŋ', 'L': 'l', 'R': 'ɹ', 'W': 'w', 'Y': 'j',
    'EM': 'm̩', 'EN': 'n̩', 'EL': 'l̩', 'NX': 'ɾ̃',
}

def arpabet_to_ipa(arpabet_list):
    """
    Convert from arpabet to ipa phonemes

    Args:
        arpabet_list (list): list of arpabet

    Returns:
        list: Contains of list chars of ipa phonemes
    """
    arpabet_phonemes = [p for p in arpabet_list if p.strip() and p not in [' ', ',', '.', '!', '?']]
    ipa_phonemes_str = " ".join([ARPABET_TO_IPA.get(p, '') for p in arpabet_phonemes])
    return [char for char in ipa_phonemes_str if char != ' ']

def phonemize_text(text):
    """
    Convert text → ARPAbet → IPA
    Args:
        text (str): text to convert into phonemes

    Returns:
        list: Contains of list chars of ipa phonemes
    """
    arpabet = g2p(text)
    return arpabet_to_ipa(arpabet)

def preprocess_dataset(dataset):
    """Add 'phonemes' column mapping"""
    dataset = dataset.map(lambda batch: {"phonemes": phonemize_text(batch["text"])})
    return dataset

In [21]:
def load_librispeech_folder(base_dir):
    """
    Load all .wav files and corresponding transcripts from a Librispeech-style folder.
    
    Args:
        base_dir (str): Path to train-clean or test-clean folder.
        
    Returns:
        datasets.Dataset: Contains 'audio' (path) and 'text' (transcript) columns.
    """
    samples = []
    
    for speaker in os.listdir(base_dir):
        spk_path = os.path.join(base_dir, speaker)
        if not os.path.isdir(spk_path):
            continue
        
        for chapter in os.listdir(spk_path):
            chap_path = os.path.join(spk_path, chapter)
            if not os.path.isdir(chap_path):
                continue
            
            trans_file = [f for f in os.listdir(chap_path) if f.endswith(".trans.txt")]
            if len(trans_file) != 1:
                continue
            trans_path = os.path.join(chap_path, trans_file[0])
            
            with open(trans_path, "r") as f:
                lines = f.readlines()
            
            trans_dict = {}
            for line in lines:
                parts = line.strip().split(" ", 1)
                if len(parts) == 2:
                    file_id, text = parts
                    trans_dict[file_id] = text
            
            for file in os.listdir(chap_path):
                if file.endswith(".wav"):
                    file_id = os.path.splitext(file)[0]
                    if file_id in trans_dict:
                        samples.append({
                            "audio": os.path.join(chap_path, file),
                            "text": trans_dict[file_id]
                        })
    
    if len(samples) == 0:
        raise ValueError(f"No valid samples found in {base_dir}")
    
    return Dataset.from_list(samples)

In [12]:
def prepare_batch(batch):
    """
    Preprocessing each batch data for fine tuning
    """
    speech, sr = sf.read(batch["audio"])
    inputs = processor(speech, sampling_rate=sr, return_attention_mask=True)

    labels = processor.tokenizer(batch["phonemes"]).input_ids

    batch["input_values"] = inputs.input_values[0]
    batch["attention_mask"] = inputs.attention_mask[0]
    batch["labels"] = labels
    return batch

## Load and Preprocessing Datasets

In [13]:
train_ds = load_librispeech_folder("LibriSpeech_final/train-clean")
test_ds  = load_librispeech_folder("LibriSpeech_final/test-clean")

train_ds = preprocess_dataset(train_ds)
test_ds  = preprocess_dataset(test_ds)

train_ds = train_ds.map(
    prepare_batch,
    remove_columns=train_ds.column_names,
    num_proc=4
)

test_ds = test_ds.map(
    prepare_batch,
    remove_columns=test_ds.column_names,
    num_proc=4
)

Parameter 'function'=<function preprocess_dataset.<locals>.<lambda> at 0x7474c630c5e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.


Map:   0%|          | 0/7051 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/7051 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/453 [00:00<?, ? examples/s]

In [22]:
print(train_ds[0])
print(len(train_ds))
print(train_ds.column_names)

print(test_ds[0])
print(len(test_ds))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Finetune Pretrained Model

In [52]:
@dataclass
class DataCollatorCTC:
    """
    Data collator for Wav2Vec2 Connectionist Temporal Classification (CTC) training.

    This collator is used with Hugging Face Trainer to prepare batches of audio features 
    and corresponding tokenized labels for training a Wav2Vec2 model with CTC loss.

    Key responsibilities:
    1. Pads input audio sequences to the same length using the processor's feature extractor.
    2. Pads tokenized labels to the same length using the tokenizer.
       - Labels are wrapped in BatchEncoding objects to ensure proper padding.
    3. Replaces pad token IDs in the labels with -100 to tell PyTorch's CTC loss
       to ignore these positions.

    Attributes:
        processor (Wav2Vec2Processor): The processor wrapping the feature extractor and tokenizer.
        padding (bool | str): Padding strategy passed to `processor.pad()` and tokenizer.pad().
            - True: pad to the longest sequence in the batch
            - 'longest': same as True
            - 'max_length': pad to the model's maximum input length
    """
    def __init__(self, processor, padding=True):
        self.processor = processor
        self.padding = padding

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        """
        Collate a batch of audio and label features.

        Args:
            features (List[Dict]): List of dataset items, each a dictionary containing:
                - "input_values": audio features (list or tensor of floats)
                - "labels": tokenized label sequence (list of ints)

        Returns:
            Dict[str, torch.Tensor]: A batch dictionary with:
                - "input_values": padded audio features (torch.Tensor)
                - "attention_mask": attention masks corresponding to audio features
                - "labels": padded labels with pad token IDs replaced by -100
        """
        input_features = [{"input_values": f["input_values"]} for f in features]
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )

        labels = []
        for f in features:
            if any(isinstance(i, list) for i in f["labels"]):
                flat_labels = [i for sub in f["labels"] for i in sub]
            else:
                flat_labels = f["labels"]
            labels.append(flat_labels)

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": labels},
            padding=True,
            return_tensors="pt"
        )

        batch["labels"] = labels_batch["input_ids"]
        batch["labels"][batch["labels"] == self.processor.tokenizer.pad_token_id] = -100

        return batch

In [64]:
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    """
    Compute CER (Character Error Rate) for Wav2Vec2 CTC.
    pred: PredictionOutput dari Trainer
    """
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred_str = processor.batch_decode(pred_ids)  # ignore -100
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [65]:
data_collator = DataCollatorCTC(processor)

training_args = TrainingArguments(
    output_dir="./lingulu_wav2vec2_pronunciation_continued_fine_tuning",
    group_by_length=False,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="steps",
    num_train_epochs=10,
    save_steps=100,
    eval_steps=100,
    save_total_limit=3,
    learning_rate=1e-5,
    logging_steps=50,
    gradient_accumulation_steps=2,
    fp16=True,
    load_best_model_at_end=True,
    resume_from_checkpoint=True,
    remove_unused_columns=False
)

In [66]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Cer
100,0.878667,0.262693,0.114911
200,0.498431,0.163200,0.050419
300,0.420933,0.142866,0.047481
400,0.394375,0.130847,0.045201
500,0.382218,0.126431,0.044807
600,0.348193,0.122916,0.043579
700,0.347102,0.118851,0.043097
800,0.388044,0.116939,0.042659
900,0.335392,0.114671,0.042527
1000,0.361582,0.115864,0.042045


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=4410, training_loss=0.33952167109837605, metrics={'train_runtime': 15717.5827, 'train_samples_per_second': 4.486, 'train_steps_per_second': 0.281, 'total_flos': 3.3953558091229377e+19, 'train_loss': 0.33952167109837605, 'epoch': 10.0})

## Export Train History

In [73]:
import pandas as pd
pd.DataFrame(trainer.state.log_history).to_csv("train_history.csv", index=False)

## Export Model and Processor

In [74]:
model.save_pretrained("./lingulu_wav2vec2_pronounciation_finetune")
processor.save_pretrained("./lingulu_wav2vec2_pronounciation_finetune")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

['./lingulu_wav2vec2_pronounciation_finetune/processor_config.json']

In [ ]:
!zip -r lingulu_wav2vec2_pronounciation_finetune.zip lingulu_wav2vec2_pronounciation_finetune/

  adding: lingulu_wav2vec2_pronounciation_finetune/ (stored 0%)
  adding: lingulu_wav2vec2_pronounciation_finetune/processor_config.json (deflated 43%)
  adding: lingulu_wav2vec2_pronounciation_finetune/vocab.json (deflated 51%)
  adding: lingulu_wav2vec2_pronounciation_finetune/added_tokens.json (deflated 10%)
  adding: lingulu_wav2vec2_pronounciation_finetune/tokenizer_config.json (deflated 69%)
  adding: lingulu_wav2vec2_pronounciation_finetune/model.safetensors